1. Importação das Bibliotecas Necessárias

Importamos as bibliotecas essenciais para manipulação de dados, visualização, pré-processamento, construção e treinamento do modelo, e tratamento de feriados.

In [ ]:
# Importação de bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Bibliotecas para pré-processamento
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score

# Bibliotecas para construção e treinamento do modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

# Biblioteca para lidar com feriados
import holidays

# Ignorar avisos para uma saída mais limpa
import warnings
warnings.filterwarnings('ignore')

2. Carregamento e Inspeção dos Dados

Carregamos o arquivo CSV contendo os dados de vendas e realizamos uma inspeção inicial para entender a estrutura dos dados.

In [ ]:
# Definir o caminho do arquivo CSV
csv_path = '../data/dados_transacao_26173.csv'  # Atualize o caminho conforme necessário

# Carregar o CSV
df = pd.read_csv(csv_path)

# Converter a coluna 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')

# Exibir as primeiras linhas do dataset
print("Primeiras linhas do dataset:")
display(df.head())

# Informações gerais sobre o dataset
print("\nInformações gerais do dataset:")
print(df.info())

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
display(df.describe())

3. Pré-processamento dos Dados

Agregamos os dados por dia, criamos um range completo de datas para garantir que não haja datas faltantes e lidamos com valores ausentes. Também adicionamos lags das vendas anteriores para enriquecer o conjunto de features.

In [ ]:
# Agregar os dados por dia
daily_df = df.groupby('Data').agg({
    'ValorUnitario': 'mean',  # Média do valor unitário por dia
    'Quantidade': 'sum'       # Quantidade total vendida por dia
}).reset_index()

# Exibir as primeiras linhas do dataframe agregado
print("\nPrimeiras linhas do dataframe agregado por dia:")
display(daily_df.head())

# Criar um range completo de datas
all_dates = pd.date_range(start=daily_df['Data'].min(), end=daily_df['Data'].max(), freq='D')
daily_df = daily_df.set_index('Data').reindex(all_dates).reset_index()
daily_df.rename(columns={'index': 'Data'}, inplace=True)

# Preencher valores ausentes
daily_df['Quantidade'].fillna(0, inplace=True)
daily_df['ValorUnitario'].interpolate(method='linear', inplace=True)
daily_df['ValorUnitario'].fillna(method='bfill', inplace=True)

# Confirmar a ausência de valores faltantes
print("\nValores faltantes após tratamento:")
print(daily_df.isnull().sum())

# Adicionar lags das vendas anteriores
for lag in range(1, 8):
    daily_df[f'Quantidade_Lag{lag}'] = daily_df['Quantidade'].shift(lag)
    daily_df[f'ValorUnitario_Lag{lag}'] = daily_df['ValorUnitario'].shift(lag)

# Adicionar lags adicionais até 60 dias
for lag in range(8, 60):  # Continuar do lag 8 ao 60
    daily_df[f'Quantidade_Lag{lag}'] = daily_df['Quantidade'].shift(lag)
    daily_df[f'ValorUnitario_Lag{lag}'] = daily_df['ValorUnitario'].shift(lag)

# Preencher valores ausentes resultantes dos lags com zeros
daily_df.fillna(0, inplace=True)

daily_df['Quantidade'] = np.log1p(daily_df['Quantidade'])  # log1p é log(x+1) para lidar com zeros


# Exibir as primeiras linhas com os lags
print("\nDataframe com lags adicionados:")
display(daily_df.head())

4. Engenharia de Features

Adicionamos features derivadas da data que podem ajudar na previsão, como dia, mês, ano, dia da semana e feriados.

In [ ]:
# Adicionar features derivadas da data
daily_df['Dia'] = daily_df['Data'].dt.day
daily_df['Mes'] = daily_df['Data'].dt.month
daily_df['Ano'] = daily_df['Data'].dt.year
daily_df['DiaDaSemana'] = daily_df['Data'].dt.dayofweek
daily_df['Quantidade_Rolling_Mean'] = daily_df['Quantidade'].rolling(window=7, min_periods=1).mean()
daily_df['ValorUnitario_Rolling_Mean'] = daily_df['ValorUnitario'].rolling(window=7, min_periods=1).mean()
daily_df['SemanaDoAno'] = daily_df['Data'].dt.isocalendar().week

# Inicializar feriados do Brasil
br_holidays = holidays.Brazil()

# Adicionar coluna de feriado: 1 se feriado, 0 caso contrário
daily_df['Feriado'] = daily_df['Data'].isin(br_holidays).astype(int)

# Exibir as primeiras linhas com as novas features
print("\nDataframe com features adicionais:")
display(daily_df.head())

5. Divisão dos Dados

Dividimos os dados em conjuntos de treino inicial, treino continuado/validação e teste conforme a estratégia descrita.

In [ ]:
# Definir as datas de corte
train_end = pd.to_datetime('2022-12-31')
continue_train_end = pd.to_datetime('2023-12-31')
test_start = pd.to_datetime('2024-01-01')
test_end = pd.to_datetime('2024-03-30')

# Separar os dados
train_df = daily_df[daily_df['Data'] <= train_end].reset_index(drop=True)
continue_train_df = daily_df[(daily_df['Data'] > train_end) & (daily_df['Data'] <= continue_train_end)].reset_index(drop=True)
test_df = daily_df[(daily_df['Data'] >= test_start) & (daily_df['Data'] <= test_end)].reset_index(drop=True)

# Exibir o número de registros em cada conjunto
print(f"\nTreino Inicial: {train_df.shape[0]} registros")
print(f"Treino Continuado: {continue_train_df.shape[0]} registros")
print(f"Teste: {test_df.shape[0]} registros")

6. Escalonamento dos Dados

Escalonamos os dados para melhorar o desempenho do modelo LSTM. Utilizamos o MinMaxScaler para normalizar as features e os targets entre 0 e 1.

In [ ]:
# Definir features e targets
feature_cols = ['Dia', 'Mes', 'Ano', 'DiaDaSemana', 'Feriado', 'SemanaDoAno'] + \
               [f'Quantidade_Lag{lag}' for lag in range(1, 15)] + \
               [f'ValorUnitario_Lag{lag}' for lag in range(1, 15)]

target_cols = ['ValorUnitario', 'Quantidade']

# Inicializar scalers
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Ajustar scalers nos dados de treino inicial
feature_scaler.fit(train_df[feature_cols])
target_scaler.fit(train_df[target_cols])

# Escalonar dados de treino inicial
train_features_scaled = feature_scaler.transform(train_df[feature_cols])
train_targets_scaled = target_scaler.transform(train_df[target_cols])

# Escalonar dados de treino continuado
continue_train_features_scaled = feature_scaler.transform(continue_train_df[feature_cols])
continue_train_targets_scaled = target_scaler.transform(continue_train_df[target_cols])

# Escalonar dados de teste
test_features_scaled = feature_scaler.transform(test_df[feature_cols])
test_targets_scaled = target_scaler.transform(test_df[target_cols])

# Exibir uma amostra dos dados escalonados de treino
print("\nAmostra dos dados escalonados de treino:")
display(pd.DataFrame(train_features_scaled, columns=feature_cols).head())

7. Criação de Sequências para o LSTM

Criamos sequências temporais dos dados para treinar o modelo LSTM. Cada sequência de entrada consiste em um número fixo de dias anteriores (por exemplo, 30 dias) e a saída é o valor do dia seguinte.

In [ ]:
# Função para criar sequências
def create_sequences(features, targets, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(features)):
        X.append(features[i-seq_length:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

SEQ_LENGTH = 60  # Número de dias usados para prever

# Criar sequências de treino
X_train, y_train = create_sequences(train_features_scaled, train_targets_scaled, SEQ_LENGTH)
print(f"\nForma de X_train: {X_train.shape}, y_train: {y_train.shape}")

# Criar sequências de treino continuado
X_continue_train, y_continue_train = create_sequences(continue_train_features_scaled, continue_train_targets_scaled, SEQ_LENGTH)
print(f"Forma de X_continue_train: {X_continue_train.shape}, y_continue_train: {y_continue_train.shape}")

# Criar sequências de teste
X_test, y_test = create_sequences(test_features_scaled, test_targets_scaled, SEQ_LENGTH)
print(f"Forma de X_test: {X_test.shape}, y_test: {y_test.shape}")

8. Construção do Modelo LSTM

Definimos a arquitetura do modelo LSTM com camadas adicionais e dropout para melhorar a capacidade de generalização.

In [ ]:
# Construção do modelo LSTM avançado com mais camadas

# Modelo ajustado com mais camadas e regularização
model = Sequential()

# Primeira camada LSTM
model.add(LSTM(512, return_sequences=True, input_shape=(SEQ_LENGTH, len(feature_cols)), 
               dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))
# Segunda camada LSTM
model.add(LSTM(256, return_sequences=True, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))
# Terceira camada LSTM
model.add(LSTM(128, return_sequences=False, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))

# Camadas densas para processar as saídas da LSTM
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))

# Camada de saída
model.add(Dense(len(target_cols)))

# Compilação do modelo com taxa de aprendizado reduzida
optimizer = Adam(learning_rate=0.0005)  # Taxa de aprendizado inicial ajustada
model.compile(optimizer=optimizer, loss='mse')

# Exibir o resumo do modelo
print("\nResumo do modelo LSTM avançado:")
model.summary()


9. Treinamento do Modelo

Treinamos o modelo inicialmente com os dados de treino (até 31/12/2022) e depois continuamos o treinamento com os dados de 2023.

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=80, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

# Criar múltiplas janelas para comparação
for seq_length in [30, 60, 90]:
    X_train, y_train = create_sequences(train_features_scaled, train_targets_scaled, seq_length)
    print(f"Treinamento com janela de {seq_length} dias: X_train.shape = {X_train.shape}")

# Treinar o modelo com os dados de treino inicial
history = model.fit(
    X_train, y_train,
    epochs=200,  # Mantendo 200 épocas para garantir tempo suficiente de treinamento
    batch_size=64,  # Lote ajustado para balancear memória e desempenho
    validation_split=0.1,  # Separar 10% dos dados de treino para validação
    callbacks=[early_stop, reduce_lr],
    verbose=1  # Mostrar logs detalhados
)

# Plotar a perda de treino e validação
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo durante o Treinamento Inicial')
plt.xlabel('Épocas')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

In [ ]:
# Treinamento contínuo para ajustar novas tendências de 2023
history_continue = model.fit(
    X_continue_train, y_continue_train,
    epochs=200,  # Período de ajuste mais curto
    batch_size=64,  # Mantendo o mesmo batch size
    validation_split=0.1,  # Validação consistente com o treinamento inicial
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Plotar a perda de treino e validação durante a continuação do treinamento
plt.figure(figsize=(12,6))
plt.plot(history_continue.history['loss'], label='Treino Continuado')
plt.plot(history_continue.history['val_loss'], label='Validação Continuada')
plt.title('Perda do Modelo durante a Continuação do Treinamento')
plt.xlabel('Épocas')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

Visualização da Perda durante o Treinamento:

In [ ]:
# Plotar a perda de treino e validação
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo durante o Treinamento')
plt.xlabel('Épocas')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()


10. Previsão e Avaliação do Modelo

Realizamos previsões para o conjunto de teste, invertemos o escalonamento dos dados e avaliamos o desempenho do modelo utilizando métricas como MSE e MAE.

In [ ]:
# Verificar o número de registros em test_df
print(f"Número de registros em test_df: {len(test_df)}")
print(f"Tamanho da sequência (SEQ_LENGTH): {SEQ_LENGTH}")

# Garantir que test_df tenha registros suficientes
if len(test_df) < SEQ_LENGTH:
    missing_records = SEQ_LENGTH - len(test_df)
    print(f"Expandindo test_df com {missing_records} registros do treino contínuo.")
    test_df = pd.concat([continue_train_df.tail(missing_records), test_df]).reset_index(drop=True)
    continue_train_df = continue_train_df.iloc[:-missing_records].reset_index(drop=True)

# Recriar sequências de teste
X_test, y_test = create_sequences(test_features_scaled, test_targets_scaled, SEQ_LENGTH)
print(f"Forma de X_test: {X_test.shape}, y_test: {y_test.shape}")

# Realizar previsões somente se X_test não estiver vazio
if X_test.shape[0] > 0:
    predictions = model.predict(X_test)

    # Inverter o escalonamento das previsões e dos valores reais
    predictions_inverse = target_scaler.inverse_transform(predictions)
    y_test_inverse = target_scaler.inverse_transform(y_test)

    # Garantir valores não negativos após inversão
    predictions_inverse = np.clip(predictions_inverse, a_min=0, a_max=None)
    y_test_inverse = np.clip(y_test_inverse, a_min=0, a_max=None)

    # Criar DataFrame para comparação
    comparison_df = test_df.iloc[SEQ_LENGTH:].copy().reset_index(drop=True)
    comparison_df['ValorUnitario_Previsto'] = predictions_inverse[:, 0]
    comparison_df['Quantidade_Prevista'] = predictions_inverse[:, 1]

    # Calcular métricas
    mse_valor = mean_squared_error(y_test_inverse[:, 0], predictions_inverse[:, 0])
    mae_valor = mean_absolute_error(y_test_inverse[:, 0], predictions_inverse[:, 0])
    mse_quant = mean_squared_error(y_test_inverse[:, 1], predictions_inverse[:, 1])
    mae_quant = mean_absolute_error(y_test_inverse[:, 1], predictions_inverse[:, 1])
    r2_valor = r2_score(y_test_inverse[:, 0], predictions_inverse[:, 0])
    r2_quant = r2_score(y_test_inverse[:, 1], predictions_inverse[:, 1])

    # Exibir métricas
    print(f"Valor Unitário - R²: {r2_valor:.4f}, MSE: {mse_valor:.4f}, MAE: {mae_valor:.4f}")
    print(f"Quantidade Vendida - R²: {r2_quant:.4f}, MSE: {mse_quant:.4f}, MAE: {mae_quant:.4f}")
else:
    print("Erro: X_test ainda está vazio. Verifique os dados de entrada e o SEQ_LENGTH.")


11. Visualização dos Resultados

Visualizamos as previsões comparadas com os valores reais para entender o desempenho do modelo ao longo do tempo.

In [ ]:
# Plotagem - Real vs Previsto
plt.figure(figsize=(14,10))

# Valor Unitário
plt.subplot(2,1,1)
plt.plot(comparison_df['Data'], comparison_df['ValorUnitario'], label='Real', color='blue')
plt.plot(comparison_df['Data'], comparison_df['ValorUnitario_Previsto'], label='Previsto', color='orange', linestyle='dashed')
plt.title('Valor Unitário - Real vs Previsto')
plt.xlabel('Data')
plt.ylabel('Valor Unitário')
plt.legend()

# Quantidade Vendida
plt.subplot(2,1,2)
plt.plot(comparison_df['Data'], comparison_df['Quantidade'], label='Real', color='green')
plt.plot(comparison_df['Data'], comparison_df['Quantidade_Prevista'], label='Previsto', color='red', linestyle='dashed')
plt.title('Quantidade Vendida - Real vs Previsto')
plt.xlabel('Data')
plt.ylabel('Quantidade Vendida')
plt.legend()

plt.tight_layout()
plt.show()

12. Conclusão

Com base nas métricas e visualizações, podemos avaliar a precisão do modelo e identificar áreas para melhorias futuras.